In [131]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# define the base url needed to create the file url.
base_url = r"https://www.sec.gov"

cik_number = "320193"

# convert a normal url to a document url
normal_url = r"https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/0000320193-20-000096.txt"
normal_url = normal_url.replace('0000320193-20-000096','').replace('.txt','/index.json')

# define a url that leads to a 10k document landing page
documents_url = r"https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/index.json"
print(documents_url)

# request the url and decode it.
content = requests.get(documents_url).json()

for file in content['directory']['item']:
    
    # Grab the filing summary and create a new url leading to the file so we can download it.
    if file['name'] == 'FilingSummary.xml':

        xml_summary = base_url + content['directory']['name'] + "/" + file['name']
        
        print('-' * 100)
        print('File Name: ' + file['name'])
        print('File Path: ' + xml_summary)


https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/index.json
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/FilingSummary.xml


In [132]:
base_url = xml_summary.replace('FilingSummary.xml', '')

# print(base_url)

# request and parse the content
content = requests.get(xml_summary).content
soup = BeautifulSoup(content, 'lxml')         # Format to xml

# print(soup)
# find the 'myreports' tag because this contains all the individual reports submitted.
reports = soup.find('myreports')

# I want a list to store all the individual components of the report, so create the master list.
master_reports = []

# loop through each report in the 'myreports' tag but avoid the last one as this will cause an error.
for report in reports.find_all('report')[:-1]:

    # let's create a dictionary to store all the different parts we need.
    report_dict = {}
    report_dict['name_short'] = report.shortname.text
    report_dict['name_long'] = report.longname.text
    report_dict['position'] = report.position.text
    report_dict['category'] = report.menucategory.text
    report_dict['url'] = base_url + report.htmlfilename.text

    # append the dictionary to the master list.
    master_reports.append(report_dict)

    # print the info to the user.
    print('-'*100)
    # if (report.htmlfilename.text == "R11.htm"):
    print(base_url + report.htmlfilename.text)
    print(report.longname.text)
    print(report.shortname.text)
    print(report.menucategory.text)
    print(report.position.text)

----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R1.htm
0001001 - Document - Cover Page
Cover Page
Cover
1
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R2.htm
1001002 - Statement - CONSOLIDATED STATEMENTS OF OPERATIONS
CONSOLIDATED STATEMENTS OF OPERATIONS
Statements
2
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R3.htm
1002003 - Statement - CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME
CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME
Statements
3
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R4.htm
1003004 - S

In [133]:
# create the list to hold the statement urls
# statements_url = []

# for report_dict in master_reports:
    
#     # define the statements we want to look for.
#     item1 = r"CONSOLIDATED STATEMENTS OF OPERATIONS"
#     item2 = r"CONSOLIDATED BALANCE SHEETS"
#     item3 = r"CONSOLIDATED STATEMENTS OF CASH FLOWS"
#     # item4 = r"Consolidated Statements of Stockholder's (Deficit) Equity"
    
#     # store them in a list.
#     report_list = [item1, item2, item3]
    
#     # if the short name can be found in the report list.
#     if report_dict['name_short'] in report_list:
        
#         # print some info and store it in the statements url.
#         print('-'*100)
#         print(report_dict['name_short'])
#         print(report_dict['url'])
        
#         statements_url.append(report_dict['url'])

In [134]:
# create the list to hold the statement urls
statements_url = []

statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019318000007/R7.htm')     # Q1 2017 Consolidated Statement of Cash Flows
statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019318000070/R7.htm')     # Q2 2018 Consolidated Statement of Cash Flows
statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019318000100/R7.htm')     # Q3 2018 Consolidated Statement of Cash Flows
statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019318000145/R9.htm')     # Y 2018 Consolidated Statement of Cash Flows

statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019319000010/R7.htm')     # Q1 2018 Consolidated Statement of Cash Flows
statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019319000066/R7.htm')     # Q2 2019 Consolidated Statement of Cash Flows
statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019319000076/R7.htm')     # Q3 2019 Consolidated Statement of Cash Flows
statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/R7.htm')     # Y 2019 Consolidated Statement of Cash Flows

statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019320000010/R7.htm')     # Q1 2019 Consolidated Statement of Cash Flows
statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019320000052/R7.htm')     # Q2 2020 Consolidated Statement of Cash Flows
statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019320000062/R7.htm')     # Q3 2020 Consolidated Statement of Cash Flows
statements_url.append('https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/R7.htm')     # Y 2020 Consolidated Statement of Cash Flows

In [135]:
# let's assume we want all the statements in a single data set.
statements_data = []

# loop through each statement url
for statement in statements_url:

    # define a dictionary that will store the different parts of the statement.
    statement_data = {}
    statement_data['headers'] = []
    statement_data['sections'] = []
    statement_data['data'] = []
    
    # request the statement file content
    content = requests.get(statement).content
    report_soup = BeautifulSoup(content, 'html')

    # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
    for index, row in enumerate(report_soup.table.find_all('tr')):
        
        # first let's get all the elements.
        cols = row.find_all('td')
        
        # if it's a regular row and not a section or a table header
        if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
            reg_row = [ele.text.strip() for ele in cols]
            statement_data['data'].append(reg_row)
            
        # if it's a regular row and a section but not a table header
        elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            sec_row = cols[0].text.strip()
            statement_data['sections'].append(sec_row)
            
        # finally if it's not any of those it must be a header
        elif (len(row.find_all('th')) != 0):            
            hed_row = [ele.text.strip() for ele in row.find_all('th')]
            statement_data['headers'].append(hed_row)
            
        else:            
            print('We encountered an error.')

    # append it to the master list.
    statements_data.append(statement_data)
    print(statement_data)

{'headers': [['CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited) - USD ($) $ in Millions', '3 Months Ended'], ['Dec. 30, 2017', 'Dec. 31, 2016']], 'sections': ['Statement of Cash Flows [Abstract]', 'Operating activities:', 'Adjustments to reconcile net income to cash generated by operating activities:', 'Changes in operating assets and liabilities:', 'Investing activities:', 'Financing activities:', 'Supplemental cash flow disclosure:'], 'data': [['Cash and cash equivalents, beginning of the period', '$ 20,289', '$ 20,484'], ['Net income', '20,065', '17,891'], ['Depreciation and amortization', '2,745', '2,987'], ['Share-based compensation expense', '1,296', '1,256'], ['Deferred income tax expense/(benefit)', '(33,737)', '1,452'], ['Other', '(11)', '(274)'], ['Accounts receivable, net', '(5,570)', '1,697'], ['Inventories', '434', '(580)'], ['Vendor non-trade receivables', '(9,660)', '(375)'], ['Other current and non-current assets', '(197)', '(1,446)'], ['Accounts payable', '1

In [136]:
pd.DataFrame(statement_data['data'])


,0,1,2,3
0,"Cash, cash equivalents and restricted cash, be...","$ 50,224","$ 25,913","$ 20,289"
1,Net income,"57,411","55,256","59,531"
2,Depreciation and amortization,"11,056","12,547","10,903"
3,Share-based compensation expense,"6,829","6,068","5,340"
4,Deferred income tax benefit,(215),(340),"(32,590)"
5,Other,(97),(652),(444)
6,"Accounts receivable, net","6,917",245,"(5,322)"
7,Inventories,(127),(289),828
8,Vendor non-trade receivables,"1,553","2,931","(8,010)"
9,Other current and non-current assets,"(9,588)",873,(423)


In [ ]:
d = {}
# [i[‘sections’] for i in statements_data]
for i in statements_data:
  for j in i['data']:
    if j[0] not in d.keys():
      d[j[0]] = [j[1]]
    else:
      d[j[0]].append(j[1])
d

In [143]:
dd = {}
for i in d:
  if len(d[i]) == 12:
    dd[i] = d[i]

In [138]:
# [statement_data['headers'][1][0] for statement_data in statements_data]
# feature_data = {}
# features = []
# for feature in statements_data[0]['data']:
#   features.append(feature[0])
#   feature_data[feature[0]] = [];


# for stmt_data in statements_data:
#   for feat in stmt_data['data']:
#     feature_data[feat].append(feat[1])
# feature_data

# for statement in statements_data:
#   for i in range(len(statement['data'])):
#     print(statement['data'][i])
    # feature_data.fromkeys()
# features
# pd.DataFrame([stmt_data['data'] for stmt_data in statements_data])


In [139]:
# income_headers
# income_headers = []
# income_features = []
# income_features_data = [[]]
# income_sections = []
# for statement_data in statements_data:
#   income_headers.append((statement_data)['headers'][1][0])
#   income_data.append(statement_data['data'])
#   for feature in (statement_data)['data']:
#     income_features_data.append(feature[1])

# for feature in (statements_data[0])['data']:
#   income_features.append(feature[0])
  
# income_features


In [140]:
income_df = pd.DataFrame(income_data)

income_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
0,"Cash, cash equivalents and restricted cash, be...","$ 50,224","$ 25,913","$ 20,289",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Net income,"57,411","55,256","59,531",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Depreciation and amortization,"11,056","12,547","10,903",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,Share-based compensation expense,"6,829","6,068","5,340",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Deferred income tax benefit,(215),(340),"(32,590)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,"[Cash, cash equivalents and restricted cash, b...","[Net income, 55,256, 59,531, 48,351]","[Depreciation and amortization, 12,547, 10,903...","[Share-based compensation expense, 6,068, 5,34...","[Deferred income tax expense/(benefit), (340),...","[Other, (652), (444), (166)]","[Accounts receivable, net, 245, (5,322), (2,093)]","[Inventories, (289), 828, (2,723)]","[Vendor non-trade receivables, 2,931, (8,010),...","[Other current and non-current assets, 873, (4...","[Accounts payable, (1,923), 9,175, 8,966]","[Deferred revenue, (625), (3), (593)]","[Other current and non-current liabilities, (4...","[Cash generated by operating activities, 69,39...","[Purchases of marketable securities, (39,630),...",[Proceeds from maturities of marketable securi...,"[Proceeds from sales of marketable securities,...","[Payments for acquisition of property, plant a...",[Payments made in connection with business acq...,"[Purchases of non-marketable securities, (1,00...","[Proceeds from non-marketable securities, 1,63...","[Other, (1,078), (745), (124)]",[Cash generated by/(used in) investing activit...,"[Proceeds from issuance of common stock, 781, ...",[Payments for taxes related to net share settl...,[Pa

In [141]:
# Put the data in a DataFrame
income_df = pd.DataFrame(income_data)

# Display
print('-'*100)
print('Before Reindexing')
print('-'*100)
display(income_df.head())

# Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
income_df.index = income_df[0]
income_df.index.name = 'Category'
income_df = income_df.drop(0, axis = 1)

# Display
print('-'*100)
print('Before Regex')
print('-'*100)
display(income_df.head())

# Get rid of the '$', '(', ')', and convert the '' to NaNs.
income_df = income_df.replace('[\$,)]','', regex=True )\
                     .replace( '[(]','-', regex=True)\
                     .replace( '', 'NaN', regex=True)

# Display
print('-'*100)
print('Before type conversion')
print('-'*100)
display(income_df.head())

# everything is a string, so let's convert all the data to a float.
income_df = income_df.astype(float)

# Change the column headers
income_df.columns = income_header

# Display
print('-'*100)
print('Final Product')
print('-'*100)

# show the df
income_df

# drop the data in a CSV file if need be.
# income_df.to_csv('income_state.csv')

----------------------------------------------------------------------------------------------------
Before Reindexing
----------------------------------------------------------------------------------------------------


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
0,"Cash, cash equivalents and restricted cash, be...","$ 50,224","$ 25,913","$ 20,289",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Net income,"57,411","55,256","59,531",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Depreciation and amortization,"11,056","12,547","10,903",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,Share-based compensation expense,"6,829","6,068","5,340",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Deferred income tax benefit,(215),(340),"(32,590)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


----------------------------------------------------------------------------------------------------
Before Regex
----------------------------------------------------------------------------------------------------


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
Category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Cash, cash equivalents and restricted cash, beginning balances","$ 50,224","$ 25,913","$ 20,289",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Net income,"57,411","55,256","59,531",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Depreciation and amortization,"11,056","12,547","10,903",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Share-based compensation expense,"6,829","6,068","5,340",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Deferred income tax benefit,(215),(340),"(32,590)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


----------------------------------------------------------------------------------------------------
Before type conversion
----------------------------------------------------------------------------------------------------


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
Category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Cash, cash equivalents and restricted cash, beginning balances",50224,25913,20289,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Net income,57411,55256,59531,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Depreciation and amortization,11056,12547,10903,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Share-based compensation expense,6829,6068,5340,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Deferred income tax benefit,-215,-340,-32590,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


ValueError: ignored

In [ ]:
income_df.transpose()



In [ ]:
income_df.T.index

In [ ]:
pd.to_datetime(income_df.T.index)

In [ ]:
income_df_new = income_df.T
income_df_new.index = pd.to_datetime(income_df_new.index)

income_df_new

In [ ]:
income_df_new = income_df_new.dropna(axis=1)

income_df_new


In [ ]:
!pip install yfinance

In [ ]:
# income_df_new.mean(axis=0)

import yfinance as yf

tickers = ["AAPL"] # Tesla stock and Bitcoin in USD

start_date = '2010-01-01'
end_date = '2021-01-01'

In [ ]:
panel_data = yf.download(tickers=tickers, start=start_date, end=end_date)[['Adj Close']].dropna(axis=0)
panel_data

In [ ]:
# obtain percentage change for each stock to use as the y variable in training
panel_data['AAPL_pct_change'] = panel_data['Adj Close'].pct_change()

# panel_data.dropna(axis=0)

#ignore first row due to NaN
panel_data = panel_data.iloc[1:, :]
panel_data

In [ ]:
from datetime import timedelta
import numpy as np
# obtain dates of income_df (they are the indices)
dates = income_df_new.index
# subtract one day from each day to obtain the Friday's instead of the saturdays
dates = dates - timedelta(days=1)
# obtain the one year before the minimum date value in financial statement
new_date = dates.min() - timedelta(days=366) # subtract 90 when you do quarterlies
# append new date to our dates
new_dates = np.append(dates, np.datetime64(new_date))
# slice out the desired dates in our pct_change df
panel_data.loc[pd.DatetimeIndex(new_dates), :]

In [ ]:


# Make new dataframe of relevant stock adjusted close prices (2020-09-26, 2019-09-28, 2018-09-29, 2017-09-29)
consolidated_df = panel_data.loc[new_dates, :]


consolidated_df['AAPL_pct_change'] = consolidated_df[['Adj Close']].pct_change(-1)
consolidated_df = consolidated_df.dropna(axis = 0)
consolidated_df

In [ ]:
income_df_new['Ann_pct_change'] = consolidated_df['AAPL_pct_change']

income_df_new